# Cohen's Kappa Score 

The goal is to calculate the Kappa Score to judge the intercoder reliability and agreement between our three coders Frenard, Sally and Trixy. The tagged labels by these three coders, for each file, is merged.

Taking Sally's coding as the basis, the Kappa score is calculated between Sally's and Frenard's results and then Sally's and Trixy's results and then we observe the agreement between Sally and Frenard and then Sally Trixy.

While calculating the score, following considerations were made: <br>
1. If both the coders labelled the data as 'U', then that row is ignored.
2. If one of the coder didn't label the data and other coder labelled it as 'U', that row is ignored.
3. If a coder tagged multiple labels: <br>
    a. match it with the label given by other coder, and the first label matched is considered. <br>
    b. if there was no match, the first label for both coders is considered.

In [1]:
import glob
import numpy as np
import pandas as pd  

In [2]:
df_kappa = pd.DataFrame(columns=['File','Frenard_Similarity', 'Trixy_Similarity', 'Frenard_Difference', 'Trixy_Difference','Frenard_Po','Trixy_Po','Frenard_Pe','Trixy_Pe','Frenard_Kp','Trixy_Kp', 'Frenard_Agreement', 'Trixy_Agreement'])
df_kappa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 13 columns):
File                  0 non-null object
Frenard_Similarity    0 non-null object
Trixy_Similarity      0 non-null object
Frenard_Difference    0 non-null object
Trixy_Difference      0 non-null object
Frenard_Po            0 non-null object
Trixy_Po              0 non-null object
Frenard_Pe            0 non-null object
Trixy_Pe              0 non-null object
Frenard_Kp            0 non-null object
Trixy_Kp              0 non-null object
Frenard_Agreement     0 non-null object
Trixy_Agreement       0 non-null object
dtypes: object(13)
memory usage: 0.0+ bytes


### 1. Agreement between Frenard and Sally

In [3]:
files = [f for f in glob.glob("MergedFiles/*.csv")]

compr_files = []

for file in files:
    if len(file.split('-'))==1:
        compr_files.append(file)
  
df_kappa_file = []
df_kappa_same = []
df_kappa_diff = []
df_kappa_po = []
df_kappa_pe = []
df_kappa_kp = []
df_kappa_agreement = []

for file in compr_files:
    df = pd.read_csv(file)
    df = df.replace(np.nan, '', regex=True)
    same = 0
    diff = 0
    label_frenard = {'T':0, 'P':0, 'O':0, 'D': 0, 'H':0, 'U':0}
    label_sally = {'T':0, 'P':0, 'O':0, 'D': 0, 'H':0, 'U':0}
    
    print(df['file'][0])
    for i in range(len(df['text'])):
        if df['file'][i]=='':
            break
        if df['label_Frenard'][i]==df['label_SALLY'][i] and df['label_Frenard'][i] == 'U':
            continue
        if len(df['label_SALLY'])==0 and df['label_Frenard'][i] == 'U':
            continue
        if len(df['label_Frenard'])==0 and df['label_SALLY'][i] == 'U':
            continue
            
        label_f = str(df['label_Frenard'][i]).replace(" ", "")
        label_f = label_f.split(',')[0]
        if len(label_f)>0:
            label_frenard[label_f]+=1
        
        label_s = str(df['label_SALLY'][i]).replace(" ", "")
        label_s = label_s.split(',')[0]
        if len(label_s)>0:
            label_sally[label_s]+=1
                    
        if label_f==label_s:
            same+=1
        else:
            diff+=1
        
    print("Frenard: "+ str(label_frenard))
    print("Sally: "+ str(label_sally))
    print("SImilarity: "+str(same))
    print("Difference: "+str(diff))
    po=same/(same+diff)
    print("Observed Proportional Agreement: "+ str(po))
    
    for k in label_frenard:
        label_frenard[k]=label_frenard[k]/(same+diff)
        
    for k in label_sally:
        label_sally[k]=label_sally[k]/(same+diff)
    
    label_prob = {'T':0, 'P':0, 'O':0, 'D': 0, 'H':0, 'U':0}
    
    for k in label_prob:
        label_prob[k] = label_frenard[k]*label_sally[k]
        
    pe = 0
    for k in label_prob:
        pe += label_prob[k]
     
    print("Hypothetical Probability of Chance Agreement: " + str(pe))
    
    kappa = (po-pe)/(1-pe)
    
    print("Cohen's Kappa: "+ str(kappa))
    
    print()
    
    df_kappa_file.append(df['file'][0])
    df_kappa_same.append(same)
    df_kappa_diff.append(diff)
    df_kappa_po.append(po)
    df_kappa_pe.append(pe)
    df_kappa_kp.append(kappa)
    
    if kappa<0.1:
        df_kappa_agreement.append('No')
    elif kappa>=0.1 and kappa<0.2:
        df_kappa_agreement.append('Slight')
    elif kappa>=0.2 and kappa<0.4:
        df_kappa_agreement.append('Fair')
    elif kappa>=0.4 and kappa<0.6:
        df_kappa_agreement.append('Moderate')
    elif kappa>=0.6 and kappa<0.8:
        df_kappa_agreement.append('Substatial')
    elif kappa>=0.8 and kappa<1:
       df_kappa_agreement.append('Near Perfect')     
    elif kappa==1:
        df_kappa_agreement.append('Perfect')

df_kappa['File']=df_kappa_file
df_kappa['Frenard_Similarity']=df_kappa_same
df_kappa['Frenard_Difference']=df_kappa_diff
df_kappa['Frenard_Po']=df_kappa_po
df_kappa['Frenard_Pe']=df_kappa_pe
df_kappa['Frenard_Kp']=df_kappa_kp
df_kappa['Frenard_Agreement']=df_kappa_agreement

ML071350662.txt
Frenard: {'T': 1, 'P': 1, 'O': 4, 'D': 7, 'H': 0, 'U': 0}
Sally: {'T': 2, 'P': 0, 'O': 3, 'D': 6, 'H': 0, 'U': 2}
SImilarity: 10
Difference: 3
Observed Proportional Agreement: 0.7692307692307693
Hypothetical Probability of Chance Agreement: 0.33136094674556216
Cohen's Kappa: 0.6548672566371682

ML12027A131.txt
Frenard: {'T': 3, 'P': 3, 'O': 5, 'D': 0, 'H': 4, 'U': 2}
Sally: {'T': 3, 'P': 1, 'O': 6, 'D': 0, 'H': 2, 'U': 5}
SImilarity: 9
Difference: 8
Observed Proportional Agreement: 0.5294117647058824
Hypothetical Probability of Chance Agreement: 0.20761245674740486
Cohen's Kappa: 0.40611353711790393

ML13182A476.txt
Frenard: {'T': 2, 'P': 3, 'O': 14, 'D': 1, 'H': 0, 'U': 1}
Sally: {'T': 1, 'P': 2, 'O': 11, 'D': 0, 'H': 0, 'U': 7}
SImilarity: 12
Difference: 9
Observed Proportional Agreement: 0.5714285714285714
Hypothetical Probability of Chance Agreement: 0.38321995464852604
Cohen's Kappa: 0.3051470588235294

ML13298A103.txt
Frenard: {'T': 0, 'P': 1, 'O': 0, 'D': 2, 'H':

### 2. Agreement between Trixy and Sally

In [4]:
files = [f for f in glob.glob("MergedFiles_KappaScore/*.csv")]

compr_files = []

for file in files:
    if len(file.split('-'))==1:
        compr_files.append(file)

df_kappa_same = []
df_kappa_diff = []
df_kappa_po = []
df_kappa_pe = []
df_kappa_kp = []
df_kappa_agreement = []

for file in compr_files:
    df = pd.read_csv(file)
    df = df.replace(np.nan, '', regex=True)
    same = 0
    diff = 0
    label_struck = {'T':0, 'P':0, 'O':0, 'D': 0, 'H':0, 'U':0}
    label_sally = {'T':0, 'P':0, 'O':0, 'D': 0, 'H':0, 'U':0}
    
    print(df['file'][0])
    for i in range(len(df['text'])):
        if df['file'][i]=='':
            break
        if df['label_struck'][i]==df['label_SALLY'][i] and df['label_struck'][i] == 'U':
            continue
        if len(df['label_SALLY'])==0 and df['label_struck'][i] == 'U':
            continue
        if len(df['label_struck'])==0 and df['label_SALLY'][i] == 'U':
            continue
            
        label_t = str(df['label_struck'][i]).replace(" ", "")
        label_t = label_t.split(',')[0]
        if len(label_t)>0:
            label_struck[label_t]+=1
        
        label_s = str(df['label_SALLY'][i]).replace(" ", "")
        label_s = label_s.split(',')[0]
        if len(label_s)>0:
            label_sally[label_s]+=1
                    
        if label_t==label_s:
            same+=1
        else:
            diff+=1
        
    print("Trixy: "+ str(label_struck))
    print("Sally: "+ str(label_sally))
    print("SImilarity: "+str(same))
    print("Difference: "+str(diff))
    po=same/(same+diff)
    print("Observed Proportional Agreement: "+ str(po))
    
    for k in label_struck:
        label_struck[k]=label_struck[k]/(same+diff)
        
    for k in label_sally:
        label_sally[k]=label_sally[k]/(same+diff)
    
    label_prob = {'T':0, 'P':0, 'O':0, 'D': 0, 'H':0, 'U':0}
    
    for k in label_prob:
        label_prob[k] = label_struck[k]*label_sally[k]
        
    pe = 0
    for k in label_prob:
        pe += label_prob[k]
     
    print("Hypothetical Probability of Chance Agreement: " + str(pe))
    
    kappa = (po-pe)/(1-pe)
    
    print("Cohen's Kappa: "+ str(kappa))
    
    print()
    
    df_kappa_same.append(same)
    df_kappa_diff.append(diff)
    df_kappa_po.append(po)
    df_kappa_pe.append(pe)
    df_kappa_kp.append(kappa)
    
    if kappa<0.1:
        df_kappa_agreement.append('No')
    elif kappa>=0.1 and kappa<0.2:
        df_kappa_agreement.append('Slight')
    elif kappa>=0.2 and kappa<0.4:
        df_kappa_agreement.append('Fair')
    elif kappa>=0.4 and kappa<0.6:
        df_kappa_agreement.append('Moderate')
    elif kappa>=0.6 and kappa<0.8:
        df_kappa_agreement.append('Substatial')
    elif kappa>=0.8 and kappa<1:
       df_kappa_agreement.append('Near Perfect')     
    elif kappa==1:
        df_kappa_agreement.append('Perfect')
    
df_kappa['File']=df_kappa_file
df_kappa['Trixy_Similarity']=df_kappa_same
df_kappa['Trixy_Difference']=df_kappa_diff
df_kappa['Trixy_Po']=df_kappa_po
df_kappa['Trixy_Pe']=df_kappa_pe
df_kappa['Trixy_Kp']=df_kappa_kp
df_kappa['Trixy_Agreement']=df_kappa_agreement

ML071350662.txt
Trixy: {'T': 2, 'P': 0, 'O': 4, 'D': 8, 'H': 0, 'U': 0}
Sally: {'T': 2, 'P': 0, 'O': 3, 'D': 6, 'H': 0, 'U': 3}
SImilarity: 11
Difference: 3
Observed Proportional Agreement: 0.7857142857142857
Hypothetical Probability of Chance Agreement: 0.32653061224489793
Cohen's Kappa: 0.6818181818181818

ML12027A131.txt
Trixy: {'T': 3, 'P': 3, 'O': 5, 'D': 0, 'H': 3, 'U': 2}
Sally: {'T': 3, 'P': 1, 'O': 6, 'D': 0, 'H': 2, 'U': 4}
SImilarity: 10
Difference: 6
Observed Proportional Agreement: 0.625
Hypothetical Probability of Chance Agreement: 0.21875
Cohen's Kappa: 0.52

ML13182A476.txt
Trixy: {'T': 2, 'P': 3, 'O': 12, 'D': 0, 'H': 0, 'U': 1}
Sally: {'T': 1, 'P': 2, 'O': 11, 'D': 0, 'H': 0, 'U': 4}
SImilarity: 12
Difference: 6
Observed Proportional Agreement: 0.6666666666666666
Hypothetical Probability of Chance Agreement: 0.4444444444444445
Cohen's Kappa: 0.39999999999999986

ML13298A103.txt
Trixy: {'T': 0, 'P': 1, 'O': 0, 'D': 2, 'H': 0, 'U': 2}
Sally: {'T': 0, 'P': 0, 'O': 0, 'D'

### 3. Comparisons between the Kappa score

In [5]:
df_kappa

,File,Frenard_Similarity,Trixy_Similarity,Frenard_Difference,Trixy_Difference,Frenard_Po,Trixy_Po,Frenard_Pe,Trixy_Pe,Frenard_Kp,Trixy_Kp,Frenard_Agreement,Trixy_Agreement
0,ML071350662.txt,10,11,3,3,0.769231,0.785714,0.331361,0.326531,0.654867,0.681818,Substatial,Substatial
1,ML12027A131.txt,9,10,8,6,0.529412,0.625000,0.207612,0.218750,0.406114,0.520000,Moderate,Moderate
2,ML13182A476.txt,12,12,9,6,0.571429,0.666667,0.383220,0.444444,0.305147,0.400000,Fair,Fair
3,ML13298A103.txt,0,0,5,5,0.000000,0.000000,0.240000,0.240000,-0.315789,-0.315789,No,No
4,ML14041A484.txt,11,8,55,63,0.166667,0.112676,0.203627,0.198572,-0.046411,-0.107178,No,No
5,ML14087A338.txt,8,4,44,72,0.153846,0.052632,0.231509,0.093144,-0.101059,-0.044674,No,No
6,wat_2000010.txt,4,4,0,0,1.000000,1.000000,0.625000,0.625000,1.000000,1.000000,Perfect,Perfect


In [6]:
df_kappa.to_csv('Results/Intercode_Reliability_KappaResults.csv', header=True)